# Toronto Neighborhoods Analysis

## Description

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

[TOC]

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.91 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.57 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.86 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.73 MB/s


## 1. Collecting Data<a name="item1"></a>

### Data Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Target: Data Frame
* 3 columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. (Borough is not null)
* More than one neighborhood can exist in one postal code area. 
    * For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
    * So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### Get data from website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'X-Varnish': '45384404 192792177, 948663651 327656200, 564936150 298613959', 'Content-Encoding': 'gzip', 'Date': 'Thu, 18 Apr 2019 14:57:44 GMT', 'X-Cache': 'cp1081 hit/3, cp2012 hit/3, cp2023 hit/6', 'Content-language': 'en', 'Server': 'mw1266.eqiad.wmnet', 'X-Cache-Status': 'hit-front', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Accept-Ranges': 'bytes', 'Last-Modified': 'Tue, 02 Apr 2019 21:01:21 GMT', 'X-Content-Type-Options': 'nosniff', 'Connection': 'keep-alive', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-Length': '15396', 'Server-Timing': 'cache;desc="hit-front"', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Type': 'text/html; charset=UTF-8', 'Set-Cookie': 'WMF

### Parse data from online

#### Soup

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    row = tr.find_all('td')
    if row:
        rows.append(row)
# rows

In [4]:
lst = []
for row in rows:
    postalcode = row[0].text.strip()
    borough = row[1].text.strip()
    neighborhood = row[2].text.strip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
# lst

#### Convert to pandas

In [5]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_yyz = pd.DataFrame(lst, columns = cols)
df_yyz.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### Group df by PostalCode

In [6]:
df_yyz = df_yyz.groupby('PostalCode').agg(
    {
          'Borough': 'first'
        , 'Neighborhood': ', '.join
    }
).reset_index()
df_yyz.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [7]:
df_yyz[df_yyz['PostalCode'] == 'M5A']

,PostalCode,Neighborhood,Borough
53,M5A,"Harbourfront, Regent Park",Downtown Toronto


### DF shape

In [8]:
df_yyz.shape

(103, 3)

## 2. Create Toranto Map <a name="item2"></a>

### Add Geo data to DataFrame

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

#### Get Geo data
* csv file from url: https://cocl.us/Geospatial_data

In [9]:
link_geo = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(link_geo)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join Geo data with previous Toronto Data

In [10]:
df_yyz = pd.merge(df_yyz, df_geo, left_on = 'PostalCode', right_on = 'Postal Code', how = 'left')
df_yyz.head()

,PostalCode,Neighborhood,Borough,Postal Code,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,M1B,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,M1C,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,M1E,43.763573,-79.188711
3,M1G,Woburn,Scarborough,M1G,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,M1H,43.773136,-79.239476


In [11]:
df_yyz.columns

Index(['PostalCode', 'Neighborhood', 'Borough', 'Postal Code', 'Latitude',
       'Longitude'],
      dtype='object')

In [12]:
df_yyz.drop(columns = ['Postal Code'])
df_yyz.head()

,PostalCode,Neighborhood,Borough,Postal Code,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,M1B,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,M1C,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,M1E,43.763573,-79.188711
3,M1G,Woburn,Scarborough,M1G,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,M1H,43.773136,-79.239476


### Toronto Map

In [13]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

map_yyz = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_yyz['Latitude'], df_yyz['Longitude'], df_yyz['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_yyz)  
    
map_yyz

The geograpical coordinates of Toronto are 43.653963, -79.387207.


## 3. Set up Foursquare API <a name="item3"></a>

#### Define Foursquare Credentials and Version

In [1]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [15]:
df_yyz.loc[0, 'Neighborhood']

'Rouge, Malvern'

Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = df_yyz.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_yyz.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_yyz.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# url

Send the GET request and examine the resutls

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb8906ef594df78c62c1577'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


And how many venues were returned by Foursquare?

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## 4. Explore the neighborhoods and segment them via Foursquare API 

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *venues_sample*.

In [23]:
# type your answer here

venues_sample = getNearbyVenues(  names=df_yyz['Neighborhood']
                                , latitudes=df_yyz['Latitude']
                                , longitudes=df_yyz['Longitude']
                               )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

#### Let's check the size of the resulting dataframe

In [24]:
print(venues_sample.shape)
venues_sample.head()

(2248, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood

In [26]:
venues_sample.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide, King, Richmond                                                                                         100
Agincourt                                                                                                          4
Agincourt North, L'Amoreaux East, Milliken, Steeles East                                                           3
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown     10
Alderwood, Long Branch                                                                                            10
Bathurst Manor, Downsview North, Wilson Heights                                                                   17
Bayview Village                                                                                                    4
Bedford Park, Lawrence Manor East                                                                                 25
Berczy Park                                        

#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(venues_sample['Venue Category'].unique())))

There are 277 uniques categories.


<a id='item3'></a>

## 5. Analyze Each Neighborhood

In [39]:
# one hot encoding
onehot_sample = pd.get_dummies(venues_sample[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
onehot_sample['Neighborhood'] = venues_sample['Neighborhood'] 

# move neighborhood column to the first column
cols = onehot_sample.columns.tolist()
fixed_columns = ['Neighborhood'] + [ col for col in cols if col != 'Neighborhood']
onehot_sample = onehot_sample[fixed_columns]

onehot_sample.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [40]:
onehot_sample.shape

(2248, 277)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
grouped_sample = onehot_sample.groupby('Neighborhood').mean().reset_index()
grouped_sample.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [43]:
grouped_sample.shape

(100, 277)

#### Let's print each neighborhood along with the top 5 most common venues

In [44]:
num_top_venues = 5

for hood in grouped_sample['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped_sample[grouped_sample['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.04
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2     Sandwich Place  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground  0.33
1    Sculpture Garden  0.33
2                Park  0.33
3   Accessories Store  0.00
4  Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store   0.2
1          Pizza Place   0.2
2             Pharmacy   0.1
3  Fried Chicken Joint   0.1
4       Sandwich Place   0.1


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_sample['Neighborhood']

for ind in np.arange(grouped_sample.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_sample.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,American Restaurant,Thai Restaurant,Bar,Bakery,Restaurant,Burger Joint,Asian Restaurant
1,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Skating Rink,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Sculpture Garden,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Pharmacy,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Beer Store,Liquor Store,German Restaurant,Curling Ice
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pool,Skating Rink,Sandwich Place,Pub,Athletics & Sports,Coffee Shop,Gym,Comfort Food Restaurant


<a id='item4'></a>

## 6. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [47]:
# set number of clusters
kclusters = 5

grouped_sample_clustering = grouped_sample.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_sample_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mergerd_sample = df_yyz

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mergerd_sample = mergerd_sample.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mergerd_sample.head() # check the last columns!

,PostalCode,Neighborhood,Borough,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",Scarborough,M1B,43.806686,-79.194353,0.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,M1C,43.784535,-79.160497,0.0,Moving Target,Bar,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,M1E,43.763573,-79.188711,0.0,Spa,Pizza Place,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Medical Center,Yoga Studio,Dog Run
3,M1G,Woburn,Scarborough,M1G,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Cedarbrae,Scarborough,M1H,43.773136,-79.239476,0.0,Athletics & Sports,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Lounge,Hakka Restaurant,Cosmetics Shop,Costume Shop


Finally, let's visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mergerd_sample['Latitude'], mergerd_sample['Longitude'], mergerd_sample['Neighborhood'], mergerd_sample['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[kclusters - 1],
        fill=True,
        fill_color=rainbow[kclusters - 1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 7. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [51]:
mergerd_sample.loc[mergerd_sample['Cluster Labels'] == 0, mergerd_sample.columns[[1] + list(range(5, mergerd_sample.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",-79.194353,0.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,"Highland Creek, Rouge Hill, Port Union",-79.160497,0.0,Moving Target,Bar,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Guildwood, Morningside, West Hill",-79.188711,0.0,Spa,Pizza Place,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Medical Center,Yoga Studio,Dog Run
3,Woburn,-79.216917,0.0,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,Cedarbrae,-79.239476,0.0,Athletics & Sports,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Lounge,Hakka Restaurant,Cosmetics Shop,Costume Shop
6,"East Birchmount Park, Ionview, Kennedy Park",-79.262029,0.0,Discount Store,Department Store,Coffee Shop,Bus Station,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,"Clairlea, Golden Mile, Oakridge",-79.284577,0.0,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Intersection,Fast Food Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
8,"Cliffcrest, Cliffside, Scarborough Village West",-79.239476,0.0,Motel,American Restaurant,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,"Birch Cliff, Cliffside West",-79.264848,0.0,Café,College Stadium,Skating Rink,General Entertainment,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,"Dorset Park, Scarborough Town Centre, Wexford ...",-79.273304,0.0,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Latin American Restaurant,Furniture / Home Store,Chinese Restaurant,Yoga Studio,Dog Run,Dessert Shop


#### Cluster 2

In [52]:
mergerd_sample.loc[mergerd_sample['Cluster Labels'] == 1, mergerd_sample.columns[[1] + list(range(5, mergerd_sample.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",-79.284577,1.0,Park,Playground,Sculpture Garden,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
20,"Silver Hills, York Mills",-79.374714,1.0,Park,Martial Arts Dojo,Cafeteria,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
23,York Mills West,-79.400049,1.0,Park,Bank,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
25,Parkwoods,-79.329656,1.0,Park,Fast Food Restaurant,Pool,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
30,"CFB Toronto, Downsview East",-79.464763,1.0,Park,Airport,Bus Stop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
40,East Toronto,-79.338106,1.0,Park,Convenience Store,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
44,Lawrence Park,-79.388790,1.0,Park,Bus Line,Swim School,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
50,Rosedale,-79.377529,1.0,Park,Trail,Playground,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
64,"Forest Hill North, Forest Hill West",-79.411307,1.0,Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
74,Caledonia-Fairbanks,-79.453512,1.0,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


#### Cluster 3

In [53]:
mergerd_sample.loc[mergerd_sample['Cluster Labels'] == 2, mergerd_sample.columns[[1] + list(range(5, mergerd_sample.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,-79.239476,2.0,Convenience Store,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
48,"Moore Park, Summerhill East",-79.383160,2.0,Tennis Court,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


#### Cluster 4

In [54]:
mergerd_sample.loc[mergerd_sample['Cluster Labels'] == 3, mergerd_sample.columns[[1] + list(range(5, mergerd_sample.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Bayview Village,-79.385975,3.0,Chinese Restaurant,Café,Japanese Restaurant,Bank,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
94,"Cloverdale, Islington, Martin Grove, Princess ...",-79.554724,3.0,Gift Shop,Bank,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


#### Cluster 5

In [55]:
mergerd_sample.loc[mergerd_sample['Cluster Labels'] == 4, mergerd_sample.columns[[1] + list(range(5, mergerd_sample.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Humber Bay, King's Mill Park, Kingsway Park So...",-79.498509,4.0,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
97,"Emery, Humberlea",-79.532242,4.0,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
